###  Photosynthetic Electron Transport Chain mathematical model (version from 2014)
With this Notebook you can reproduce the results presented in 

Oliver Ebenhöh, Geoffrey Fucile, Giovanni Finazzi, Jean‐David Rochaix and Michel Goldschmidt‐Clermont (2014)
"Short-term acclimation of the photosynthetic electron transfer chain to changing light: a mathematical model"
*Phil Trans Roy Soc B* 369 doi:10.1098/rstb.2013.0223

In [ ]:
from modelbase.ode import Model, Simulator

import numpy as np
import matplotlib.pyplot as plt

from model import get_model, calculate_pHinv

In [ ]:
# Create initial conditions
y0 = {
    "PQ": get_model().parameters["PQtot"],  # oxidised plastoquinone
    "PC": 0.0202,  # oxidised plastocyan
    "Fd": 5.0,  # oxidised ferrodoxin
    "ATP": 0.0,  # stromal concentration of ATP
    "NADPH": 0.0,  # stromal concentration of NADPH
    "H": calculate_pHinv(7.2),  # lumenal protons
    "LHC": 0.9,
}

The model is complete. Now using Simulator class we can create an object that is 'integratable'. 

## Reproduce PAM experiment (Fig 1. from the paper, upper pannel)

In [ ]:
s = Simulator(get_model())
s.initialise(y0)
s.update_parameters({"Ton": 270, "Toff": 900, "dT": 90})

t = 0
Tmax = 1800
while t < Tmax:
    # turn on the saturating pulse of light of Tflash length
    if t % s.model.get_parameter("dT") == 0:
        s.update_parameter("pfd", 5000)
        s.simulate(t + 0.8, steps=1000)
    else:
        # switch on the light except for the dark period
        # t+dT-Tflash is the time to the next flash
        if t < s.model.get_parameter("Ton") or t > s.model.get_parameter("Toff"):
            s.update_parameter("pfd", 0.0001)
        else:
            # put the actinic light
            s.update_parameter("pfd", 100)
        new_t = t + s.model.get_parameter("dT") - 0.8
        s.simulate(
            new_t, **{"atol": 1.0e-10}
        )  # I needed to make it smaller from the default, otherwise integration problems
    t = s.get_time()[-1]

In [ ]:
# Needs to be normalized s.plot_selection('Fluo')
f = max(s.get_variable("Fluo"))
plt.figure()
plt.plot(s.get_time(), s.get_variable("Fluo") / f)
plt.xlabel("time [s]")
plt.ylim(0, 1)
plt.ylabel("Fluorescence normalized to Fm")

### Supplementary FigureS1 
Simulated temporal evolution of the plastoquinone redox state (red) and the antenna cross‐section of PSII (blue) as a response to light‐induced state transitions.

In [ ]:
plt.figure()
plt.plot(
    s.get_time(),
    s.get_variable("PQred") / s.model.get_parameter("PQtot"),
    "r",
    label="reduced PQ",
)
plt.plot(
    s.get_time(), s.get_variable("ps2cs"), "b", label="relative cross-section of PSII"
)

plt.xlabel("time [s]")
plt.yticks([0, 0.25, 0.5, 0.6, 0.7, 0.8, 0.9, 1], [0, 25, 50, 60, 70, 80, 90, 100])
plt.ylabel("% of the reduced PQ")
plt.legend()

## Reproduce PAM experiment (Fig 1. from the paper, lower pannel, anoxic conditions)

In [ ]:
# Simulation with switched off oxygen, not light
s = Simulator(get_model())
s.initialise(y0)
s.update_parameters({"pfd": 0.001, "ox": False, "Ton": 270.0, "Toff": 900.0, "dT": 90.0})

t = 0
Tmax = 1800
while t < Tmax:
    # turn on the saturating pulse of light of Tflash length
    if t % s.model.get_parameter("dT") == 0:
        s.update_parameter("pfd", 5000)
        s.simulate(t + 0.8, **{"atol": 1.0e-10})
    else:
        s.update_parameter("pfd", 0.001)
        new_t = t + s.model.get_parameter("dT") - 0.8
        s.simulate(
            new_t, **{"atol": 1.0e-11}
        )  # I needed to make it smaller from the default, otherwise integration problems
    t = s.get_time()[-1]

In [ ]:
# Needs to be normalized s.plot_selection('Fluo')
f = max(s.get_variable("Fluo"))
plt.figure()
plt.plot(s.get_time(), s.get_variable("Fluo") / f)
plt.xlabel("time [s]")
plt.ylim(0, 1)
plt.ylabel("Fluorescence normalized to Fm")